<a href="https://colab.research.google.com/github/5050-ken/get_amzn_rvws/blob/main/get_amzn_rvws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import bs4         
import requests    
# import textwrap    
import time
import pandas as pd

# レビュー取得したい商品のurl指定
amzn_url = 'https://www.amazon.co.jp/%E4%B8%80%E7%95%AA%E6%90%BE%E3%82%8A-%E3%82%AD%E3%83%AA%E3%83%B3-350ml%C3%9724%E6%9C%AC/dp/B01BM9ECRE/ref=sr_1_5?keywords=%E4%B8%80%E7%95%AA%E6%90%BE%E3%82%8A&qid=1648989092&sr=8-5'

# 出力されるcsvファイル指定
output_name = 'test.csv'

 
# 全ページ分をリストにする
def get_all_reviews(url):
    rvw_list = []
    i = 1
    while True:
        print(i,'searching')
        i += 1
        res = requests.get(url)
        print(res)

        # 503エラーの場合リトライ
        if res.status_code == 503:
            print('retry')
            retry_num = 0
            while True:
                res = requests.get(url)
                retry_num += 1
                if res.status_code == 200:
                    break
                elif retry_num >= 9: #上限１０回
                    break

        # ページ情報取得
        amazon_soup = bs4.BeautifulSoup(res.text, features='lxml')

        # レビュータイトル
        rvw_title = amazon_soup.select('.review-title.a-color-base')

        # レビュー日付
        rvw_date = amazon_soup.select('.a-size-base.a-color-secondary.review-date')
        
        # スター数
        rvw_stars = amazon_soup.select('i.a-icon.a-icon-star.review-rating')
        
        # レビュー内容
        rvw_content = amazon_soup.select('.review-text')

        # 上位レビュー削除（日付と星のタグが同じ為）
        del rvw_date[0:2]
        del rvw_stars[0:2]

        # 取得レビュー格納
        for current_index, title in enumerate(rvw_title):
            rvws = []
            rvws.append(rvw_title[current_index].text)
            rvws.append(rvw_date[current_index].text)
            rvws.append(rvw_stars[current_index].text)
            rvws.append(rvw_content[current_index].text)

            rvw_list.append(rvws)
             
        # 次へボタン
        next_page = amazon_soup.select('li.a-last a')   
 
        if next_page != []:    
            next_url = 'https://www.amazon.co.jp/' + next_page[0].attrs['href']    
            url = next_url
            # 最低でも1秒は間隔をあける
            time.sleep(1)       
        else:
            print("----------- done！ -----------")
            break
 
    return rvw_list
 
if __name__ == '__main__':
     
    # URLをレビューページのものに書き換える
    new_url = amzn_url.replace('dp', 'product-reviews')

    # レビューの取得
    rvw_list = get_all_reviews(new_url)

    # csv出力
    rvw_list_df = pd.DataFrame(rvw_list)
    rvw_list_df.columns = ['title','date','stars','review_comment']
    rvw_list_df.to_csv(output_name)

1 searching
<Response [200]>
2 searching
<Response [200]>
3 searching
<Response [200]>
4 searching
<Response [200]>
5 searching
<Response [200]>
6 searching
<Response [200]>
7 searching
<Response [200]>
8 searching
<Response [200]>
9 searching
<Response [200]>
10 searching
<Response [200]>
11 searching
<Response [200]>
12 searching
<Response [200]>
13 searching
<Response [503]>
retry
14 searching
<Response [200]>
15 searching
<Response [200]>
16 searching
<Response [200]>
17 searching
<Response [200]>
18 searching
<Response [503]>
retry
19 searching
<Response [503]>
retry
20 searching
<Response [200]>
21 searching
<Response [503]>
retry
22 searching
<Response [503]>
retry
23 searching
<Response [200]>
24 searching
<Response [200]>
----------- done！ -----------


In [91]:
!pwd

/content
